Managing Ecological Tipping Points using Deep-RL

In [11]:
import numpy as np
import stable_baselines3 as sb3
import gym
import gym_conservation
from sb3_contrib import ARS
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [3]:
algo = "ARS"
env_id = "conservation-v6"
team = "team_cboettig"
agent_id = env_id + "-" + algo + "-" + team
agent_id

'conservation-v6-ARS-team_cboettig'

In [4]:
seed = 24
np.random.seed(seed)

In [5]:
env = gym.make(env_id)

In [6]:

model = ARS("MlpPolicy", env = env, verbose=0, tensorboard_log="/var/log/tensorboard/")

In [8]:
if not os.path.exists((agent_id+".zip")):
    model.learn(total_timesteps=3000000)
    model.save(agent_id)

In [12]:
model = ARS.load(agent_id)

score = evaluate_policy(model, Monitor(env), n_eval_episodes=10)
score

(168.4900547, 3.3162343961849894)

In [ ]:

sims = env.simulate(model, reps=100)
policy = env.policyfn(model, reps=10)
sims


In [8]:
import plotly.express as px
import plotly.graph_objects as go

rep0 = sims.query("rep==0")
#fig = px.line(rep0, x="time", y="state", title='rep 0')

df = rep0.melt(id_vars=['time','rep'],var_name='metric', value_name='values')
fig = px.line(df, x="time", y="values", color="metric", title='rep 0')

fig.show()

In [9]:
df = policy.query("rep < 5")
fig = px.line(df, x="state", y="action", color="rep", title='policy (5 reps)')
fig.show()

In [ ]:
import torch
#del model
torch.cuda.empty_cache()
